In [2]:
#biblioteki  <>
import numpy as np
import pandas as pd 
import yfinance as yf
import re
import time


In [ ]:
#Kreacja pliku
def kreacja_pliku():
    Nazwa_wig20 = (pd.read_html("https://www.biznesradar.pl/gielda/indeks:WIG20")[0]["Profil"]).to_list() 
    Nazwa_mwig40 = (pd.read_html("https://www.biznesradar.pl/gielda/indeks:mWIG40")[0]["Profil"]).to_list()
    Nazwa_swig80 = (pd.read_html("https://www.biznesradar.pl/gielda/indeks:sWIG80")[0]["Profil"]).to_list()
    lista_nazw = Nazwa_wig20 + Nazwa_mwig40 + Nazwa_swig80
    lista_nazw.sort()
    with open("nazwy.txt", "w", encoding="utf-8") as plik:
        plik.write("\n".join(lista_nazw) + "\n")
    print("Plik 'nazwy.txt' został utworzony.")
#Oczyszczanie 1  
def oczyszczanie_1():
    rzis.iloc[:,1:] = rzis.iloc[:,1:].replace(" ", "", regex=True) 
    bilans.iloc[:,1:] = bilans.iloc[:,1:].replace(" ", "", regex=True)
    cashflow.iloc[:,1:] = cashflow.iloc[:,1:].replace(" ", "", regex=True)
#Oczyszczanie 2
def oczyszczanie_2():
    rzis.columns = pd.to_datetime([col.split()[0] + "-12-31" if "Q" not in col else col.split("/")[0] + "-09-30" for col in rzis.columns], errors="coerce")
    bilans.columns = pd.to_datetime([col.split()[0] + "-12-31" if "Q" not in col else col.split("/")[0] + "-09-30" for col in bilans.columns], errors="coerce")
    cashflow.columns = pd.to_datetime([col.split()[0] + "-12-31" if "Q" not in col else col.split("/")[0] + "-09-30" for col in cashflow.columns], errors="coerce")
#Oczyszczanie 3
def oczyszczanie_3():
    rzis.iloc[:, 1:] = rzis.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    bilans.iloc[:, 1:] = bilans.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    cashflow.iloc[:, 1:] = cashflow.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

#---->> Początek programu <<----
#1. Kreacja pliku
#!! jęsli chcesz pobrać nazwy spółek z indeksów WIG20, mWIG40 i sWIG80, odkomentuj poniższą linię !!
#kreacja_pliku()

#2. Wczytanie nazw spółek z pliku "nazwy.txt"
#!! jeśli chcesz pobrać dane dla innych spółek, dodaj je do pliku "nazwy.txt" ręznie edytując plik !!
# Wczytanie nazw spółek z pliku "nazwy.txt"
with open("nazwy.txt", "r", encoding="utf-8") as plik:
    lista_spolek = [linia.strip() for linia in plik.readlines()]  # Usuwamy zbędne białe znaki


#3. Pobieranie i zapisywanie danych dla każdej spółki
for spolka in lista_spolek:
    match = re.search(r'\((.*?)\)', spolka)  # Pobieranie nazwy spółki
    if match:
        nazwa_spolki = match.group(1)  # Nazwa spółki
    else:
        nazwa_spolki = spolka[:3]  # Jeśli brak nazwy, używamy tickera
    
    ticker = spolka[:3]  # Ticker to pierwsze 3 znaki
    
    try:
        #3.1 Pobieranie danych z BiznesRadar
        time.sleep(1)
        rzis = pd.read_html(f"https://www.biznesradar.pl/raporty-finansowe-rachunek-zyskow-i-strat/{ticker}")[1][1:].applymap(lambda x: str(x)[:str(x).find("r/r")] if "r/r" in str(x) else x).applymap(lambda x: str(x).split("k/k")[0] if "k/k" in str(x) else str(x))
        time.sleep(1) # Oczekiwanie 60 sekund przed pobraniem kolejnych danych
        bilans = pd.read_html(f"https://www.biznesradar.pl/raporty-finansowe-bilans/{ticker}")[2][1:].applymap(lambda x: str(x)[:str(x).find("r/r")] if "r/r" in str(x) else x).applymap(lambda x: str(x).split("k/k")[0] if "k/k" in str(x) else str(x))
        time.sleep(1) # Oczekiwanie 30 sekund przed pobraniem kolejnych danych
        cashflow = pd.read_html(f"https://www.biznesradar.pl/raporty-finansowe-przeplywy-pieniezne/{ticker}")[0][1:].applymap(lambda x: str(x)[:str(x).find("r/r")] if "r/r" in str(x) else x).applymap(lambda x: str(x).split("k/k")[0] if "k/k" in str(x) else str(x)) 
        time.sleep(1)

        #3.2 Oczyszczanie danych
        #Oczyszcznie skałada się z 3 etapów jęsli chcesz któryś z nich pominąć zakomentuj go
        #oczyszczanie_1()
        #oczyszczanie_2()
        #oczyszczanie_3()
      

        #3.3 Zapis do pliku Excel
        nazwa_pliku = f"{nazwa_spolki}.xlsx"
        with pd.ExcelWriter(nazwa_pliku, engine="xlsxwriter") as writer:
            rzis.to_excel(writer, sheet_name="RZiS", index=False)
            bilans.to_excel(writer, sheet_name="Bilans", index=False)
            cashflow.to_excel(writer, sheet_name="Cashflow", index=False)

        print(f"Zapisano plik: {nazwa_pliku}")
        time.sleep(12)

    except Exception as e:
        print(f"Błąd dla spółki {nazwa_spolki} ({ticker}): {e}")
#---->> Koniec programu <<----  
print("Wszystkie pliki zostały zapisane.")

In [1]:
#Analiza Techniczna danych handlowych

# 1. Biblioteki
import numpy as np
import pandas as pd
import yfinance as yf
import os
from datetime import date

# 2. Pobieranie Tickerów
def pobierz_tickery(indeks):
    url = f"https://www.biznesradar.pl/gielda/indeks:{indeks}"
    return (pd.read_html(url)[0]["Profil"].str[:3] + ".WA").to_list()

tickery = pobierz_tickery("WIG20") + pobierz_tickery("mWIG40") + pobierz_tickery("sWIG80")

# 3. Pozyskiwanie daty
dziś = date.today()

# 4. Ustawienie folderu docelowego
folder = r"C:\Users\pjgf2\Desktop\Pythone\Index Investment Chellenge 2025\Dane zmienne"  # <- Zmień na swój folder


# Tworzenie folderu, jeśli nie istnieje
os.makedirs(folder, exist_ok=True)

# 5. Pobieranie danych tylko raz
dane = yf.download(tickery, start="2024-01-01", end=dziś)

# 6. Zapis do Excela
plik_excel = os.path.join(folder, "dane_gieldowe.xlsx")

with pd.ExcelWriter(plik_excel, engine="xlsxwriter") as writer:
    for typ in ["Volume", "Open", "Close", "High", "Low"]:
        dane[typ].to_excel(writer, sheet_name=typ)

print(f"Plik zapisano w: {plik_excel}")



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  140 of 140 completed


Plik zapisano w: C:\Users\pjgf2\Desktop\Pythone\Index Investment Chellenge 2025\Dane zmienne\dane_gieldowe.xlsx
